In [2]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163011 sha256=5b9dea05782b75e4aff351cca60308cd5106e3adc83addc4c7f3cb44d01b1dd5
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [10]:
from surprise import Dataset
from surprise.prediction_algorithms import knns
from surprise import KNNBasic
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
import pandas as pd
import numpy as np
from surprise.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [25]:
df = pd.read_csv('online_retail_II.csv')

In [26]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [27]:
df.drop(columns = ['Country'])

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0
...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0


In [34]:
customer_data = df.groupby('Customer ID')['Quantity', 'Price'].sum()

# Fill any missing values
customer_data.fillna(0, inplace=True)

# Feature scaling
scaler = StandardScaler()
customer_data_scaled = scaler.fit_transform(customer_data)

# K-means clustering
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(customer_data_scaled)
customer_data['Cluster'] = kmeans.labels_

<ipython-input-34-b3e74db4f99a>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  customer_data = df.groupby('Customer ID')['Quantity', 'Price'].sum()
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [36]:
df = df.merge(customer_data[['Cluster']], how='left', on='Customer ID')
print("Cluster Centers:")
print(scaler.inverse_transform(kmeans.cluster_centers_))

Cluster Centers:
[[  1244.80230391    418.97427715]
 [ 36364.07692308  34778.66846154]
 [238965.5          6423.185     ]
 [ 66248.40909091   3636.39181818]]


### **Cluster 0: Moderate Spenders, Moderate Purchasers**

Average Quantity: ~1,245 items

Average Price: ~$419

This cluster likely represents customers who make moderate-sized purchases at moderate frequencies. They are neither high spenders nor frequent buyers.

### **Cluster 1: High Spenders, High Purchasers**

Average Quantity: ~36,364 items

Average Price: ~$34,779

This cluster consists of customers who are high-value shoppers, making large quantities of purchases and spending significantly more compared to other clusters. They may represent bulk buyers or corporate clients.


### **Cluster 2: Infrequent Buyers, High Spenders**

Average Quantity: ~239,965 items

Average Price: ~$6,423

This cluster contains customers who make fewer purchases but spend a substantial amount on each transaction. They may represent occasional buyers who make high-value purchases.

### **Cluster 3: High Purchasers, Low Spenders**
Average Quantity: ~66,248 items


Average Price: ~$3,63

Customers in this cluster make a high number of purchases but spend relatively less on each transaction. They may include bargain hunters or customers who prioritize quantity over price.

In [1]:
from surprise import Reader, Dataset

reader = Reader(rating_scale=(df['Quantity'].min(), df['Quantity'].max()))

data = Dataset.load_from_df(df[['Customer ID', 'StockCode', 'Quantity']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=40)

# Define the model (User-Based Collaborative Filtering)
sim_options = {'name': 'cosine', 'user_based': True}
model = KNNBasic(sim_options=sim_options)

# Train the model
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Evaluate the model
accuracy.rmse(predictions)

NameError: name 'df' is not defined